In [1]:
from optiver_features import generate_test_df
from fastai.tabular.all import *

In [2]:
test_df = generate_test_df()
train_df = pd.read_csv('../input/optiver-train-features/train_with_features.csv')

In [3]:
def pred_tabular_nn(train_df, test_df):
    train_df = train_df.drop(['time_id', 'row_id'], axis=1).fillna(0)
    train_df.stock_id = train_df.stock_id.astype('category')
    cont_nn,cat_nn = cont_cat_split(train_df,  dep_var='target')
    dls = TabularPandas(train_df, [Categorify, Normalize], cat_nn, cont_nn, y_names='target').dataloaders(2048)
    test_dl = dls.test_dl(test_df.fillna(0))
    learn = tabular_learner(dls, y_range=(0,.1), layers=[1000,500,200], n_out=1, path = '../input/optiver-models/')
    res = torch.zeros(len(test_df))
    for idx in range(5):
        learn.load(f'nn_fold{idx}')
        preds, _ = learn.get_preds(dl=test_dl)
        res += preds.squeeze() / 5
    return res.numpy()

In [4]:
def pred_lgb(test_df):
    test_df = test_df.drop(['row_id', 'time_id'], axis=1)
    res = np.zeros(len(test_df))
    for idx in range(10):
        filename = f'../input/optiver-models/models/lgb_fold{idx}.pickle'
        model = pickle.load(open(filename, 'rb'))
        preds = model.predict(test_df)
        res += preds / 10
    return res

In [5]:
nn_preds = pred_tabular_nn(train_df, test_df)
lgb_preds = pred_lgb(test_df)

test_df['target']=(nn_preds*0.575+lgb_preds*0.425)
test_df[['row_id', 'target']].to_csv('submission.csv', index =False)
pd.read_csv('submission.csv').head()

,row_id,target
0,0-4,0.003316
1,0-32,0.000592
2,0-34,0.000592
